In [2]:
import settings
%matplotlib inline
import sys
sys.path.append('../../../../../cocoapi/PythonAPI')
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import skimage.io as io
import pylab
import pandas as pd
pylab.rcParams['figure.figsize'] = (10.0, 8.0)


In [2]:
annType = 'keypoints'      #specify type here
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
print ('Running demo for *%s* results.'%(annType))

Running demo for *keypoints* results.


In [3]:
import json
#initialize COCO ground truth api
dataDir='./'
dataType='val2017'
annFile = 'person_keypoints_val2017.json'
cocoGt=COCO(annFile)
print(cocoGt)

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


In [4]:
#initialize COCO detections api
resFile = 'alphapose-results_0219-fast-421-id.json'
cocoDt=cocoGt.loadRes(resFile)

Loading and preparing results...
DONE (t=0.70s)
creating index...
index created!


In [5]:
imgIds = sorted(cocoGt.getImgIds())
imgIds = imgIds[0:5000]
imgId = imgIds[np.random.randint(5000)]

In [6]:
cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds
cocoEval.params.iouType = 'keypoints'
cocoEval.evaluate()

cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *keypoints*
DONE (t=6.28s).
Accumulating evaluation results...
DONE (t=0.10s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.734
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.902
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.809
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.696
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.797
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.772
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.920
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.839
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.730
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.835


In [7]:
print(cocoEval._dts[(2299, 1)])
def set_objid_oks_dt(cocoEval):
    arrs = []
    gt_idx = 0
    for i in range(len(cocoEval.evalImgs)): 
        tmp = {}
        obj = cocoEval.evalImgs[i] 
        if obj is not None :
            obj_id = obj["image_id"] 
            gtid_list = []
            dtid_list = []
            for g in range(len(cocoEval._gts[(obj_id, 1)])):
                gtid_list.append(cocoEval._gts[(obj_id,1)][g]['id']) # modify
        #    for d in range(len(cocoEval._dts[(obj_id, 1)])):
        #        dtid_list.append(cocoEval._dts[(obj_id,1)][d]['id'])
            if obj["aRng"] == [0, 10000000000.0] : # scale all
                gtMatch = obj["gtMatches"][0].tolist()
                dtMatch = obj["dtMatches"][0].tolist()
                for j in range(len(gtid_list)):
                    keypoint = []
                    flag = 0
                    Id = gtid_list[j] # modify
                    ign_flag = obj["gtIds"].index(Id)

                    if obj["gtIgnore"][ign_flag] != 1 :
                        dt_id = obj["dtIds"]
                        if Id in dtMatch:
                            dtId = dtMatch.index(Id)
                            tmp_n = dt_id[dtId]
                            if tmp_n in gtMatch:
                                id_oks = cocoEval.ious[(obj_id, 1)][dtId, j]
                            else:
                                flag =2
                        else:
                            flag = 2
                        if flag ==2:
                            id_oks = 0
                            continue
                        for m in range(len(cocoEval._dts[(obj_id, 1)])):
                            if cocoEval._dts[(obj_id, 1)][m]['id'] == tmp_n:
                                keypoint = str(cocoEval._dts[(obj_id, 1)][m]['keypoints'])
                                dtscores = float(cocoEval._dts[(obj_id, 1)][m]['score'])
                                
                                break
                        arrs.append(obj_id)
                        arrs.append(Id)
                        arrs.append(dtscores)
                        arrs.append(id_oks)
                        
                        arrs.append(keypoint)
                        

                    else:
                        continue
    arrs = np.array(arrs)
    arrs = arrs.reshape(-1, 5)
    arrs = arrs.tolist()
    #print(arrs)

    res = sorted(arrs, key = lambda pers:pers[3])


    df = pd.DataFrame(res, columns = ["image_id", "gt_id", "dtScores", "oks", "keypoints"])
    #df.to_csv("./Keypoint_sorted_oks_score_order_ann_ignore_2.csv", index = False)
    return res, df

matching_results, matching_results_df = set_objid_oks_dt(cocoEval)
matching_results_df.to_csv("./testing_alpha.csv", index = False)
matching_results_df

[{'bbox': [6.473819255828857, 144.39443969726562, 34.613667011260986, 116.30194091796875], 'score': 2.9782052040100098, 'id': 9974, 'area': 4025.636655697919, 'image_id': 2299, 'category_id': 1, 'idx': [0.0], 'keypoints': [24.47292709350586, 147.1635284423828, 0.9327667951583862, 27.242019653320312, 144.39443969726562, 0.9314673542976379, 20.319286346435547, 144.39443969726562, 0.9527952671051025, 32.78020477294922, 145.77899169921875, 0.9197709560394287, 13.396552085876465, 147.1635284423828, 0.9070677757263184, 38.31839370727539, 165.1626434326172, 0.8825149536132812, 9.242912292480469, 166.54718017578125, 0.8653008937835693, 41.087486267089844, 190.08447265625, 0.886196494102478, 6.473819255828857, 194.2381134033203, 0.8974536061286926, 32.78020477294922, 205.3144989013672, 0.8436359167098999, 18.93474006652832, 206.69903564453125, 0.8501332998275757, 34.16475296020508, 203.92994689941406, 0.7642657160758972, 16.165645599365234, 205.3144989013672, 0.7805755734443665, 36.933845520019

,image_id,gt_id,dtScores,oks,keypoints
0,8021,485520,2.359494686126709,0.500390467535023,"[259.2479553222656, 428.8138732910156, 0.76577..."
1,253386,1722376,1.3833022117614746,0.5008177897599442,"[451.68475341796875, 281.20654296875, 0.032896..."
2,183709,571226,2.3658406734466553,0.5034771632062718,"[440.2206726074219, 252.22174072265625, 0.7709..."
3,329455,2164354,1.3165369033813477,0.5052816938751674,"[636.1887817382812, 216.72645568847656, 0.2516..."
4,5037,1292330,1.2821412086486816,0.5053619688911782,"[211.71762084960938, 165.810791015625, 0.44757..."
5,199771,1314232,1.5318259000778198,0.5093938983846497,"[58.060829162597656, 85.9519271850586, 0.41063..."
6,2685,1298949,1.3177547454833984,0.5096255310115797,"[200.70960998535156, 164.75927734375, 0.479259..."
7,57150,1281612,1.251358985900879,0.5104771256388191,"[84.88264465332031, 46.402854919433594, 0.5810..."
8,112997,444642,1.4759697914123535,0.5136510942843769,"[70.1264877319336, 191.54685974121094, 0.03897..."
9,480275,436313,1.395983099937439,0.5143008703428774,"[525.4805908203125, 121.42571258544922, 0.6294..."


In [8]:
from sklearn.preprocessing import minmax_scale
matching_results_np = np.array(matching_results)
score_col = []
for i in range(len(matching_results_np)):
    score_col.append(matching_results_np[i][2])
print(matching_results_np[0])
minmax_matching = minmax_scale(score_col, axis = 0, copy = True)
for i in range(len(matching_results_np)):
    matching_results_np[i][2] = minmax_matching[i]
print(minmax_matching[0])
print(matching_results_np[0])
res = matching_results_np.tolist()
norm_df = pd.DataFrame(res, columns = ["image_id", "gt_id", "dtScores", "oks", "keypoints"])
norm_df.to_csv("./testing_alpha_norm.csv", index = False)

['8021' '485520' '2.359494686126709' '0.500390467535023'
 '[259.2479553222656, 428.8138732910156, 0.7657778859138489, 247.35382080078125, 408.99029541015625, 0.4781322479248047, 243.38909912109375, 408.99029541015625, 0.868638277053833, 13.435823440551758, 397.0961608886719, 0.3719925284385681, 227.53025817871094, 432.778564453125, 0.34526970982551575, 17.400535583496094, 464.49627685546875, 0.1508830189704895, 279.071533203125, 468.4609680175781, 0.24981559813022614, 21.36524772644043, 405.0256042480469, 0.2521931827068329, 239.4243927001953, 428.8138732910156, 0.13350103795528412, 84.8006362915039, 313.83721923828125, 0.04755910485982895, 283.0362548828125, 464.49627685546875, 0.28727811574935913, 45.15351867675781, 448.6374206542969, 0.04328780621290207, 172.0242919921875, 448.6374206542969, 0.0885464996099472, 96.69477081298828, 286.084228515625, 0.3875974118709564, 100.65947723388672, 294.013671875, 0.20642605423927307, 29.2946720123291, 436.7432861328125, 0.034149304032325745, 21

In [61]:
op_results = settings.csv2list('testing_open.csv')
alp_results = settings.csv2list('testing_alpha_norm.csv')

In [62]:
def finding_same_obj(op, ap, modeltype):
    # 두 결과에서 같은 id만 추출
    only_op = []
    only_ap = []
    AO_together_op = []
    A0_together_ap = []
    for i in range(len(op)):
        flag = 0
        img_id = op[i][0]

        gt_id = op[i][1]
        for j in range(len(ap)):
            aimg_id = ap[j][0]
            agt_id = ap[j][1]
            if (img_id, gt_id ) == (aimg_id, agt_id):
                flag = 0
                break
            flag = 1
        if flag == 1:
            only_op.append(op[i])
        else:
            AO_together_op.append(op[i])
           

    for i in range(len(ap)):
        flag = 0
        img_id = ap[i][0]

        gt_id = ap[i][1]
        for j in range(len(op)):
            aimg_id = op[j][0]
            agt_id = op[j][1]
            if (img_id, gt_id ) == (aimg_id, agt_id):
                flag = 0
                break
            flag = 1
        if flag == 1:
            only_ap.append(ap[i])
        else:
            A0_together_ap.append(ap[i])
            
            
    only_op_df = pd.DataFrame(only_op, columns = ["image_id", "gt_id","dtScores", "oks", "keypoints"])
    same_df_op = pd.DataFrame(AO_together_op, columns = ["image_id", "gt_id", "dtScores","oks", "keypoints"])
    only_ap_df = pd.DataFrame(only_ap, columns = ["image_id", "gt_id","dtScores", "oks", "keypoints"])
    same_df_ap = pd.DataFrame(A0_together_ap, columns = ["image_id", "gt_id", "dtScores","oks", "keypoints"])
    
    return only_op_df, same_df_op, only_ap_df, same_df_ap

only_op, same_opap_op, only_ap, same_opap_ap = finding_same_obj(op_results, alp_results, 'maximize')

In [63]:
only_op.to_csv('./score_maximize_Only_in_OpenPose.csv', index = False)
same_opap_op.to_csv('./score_maximize_Same_with_APOP_ver_op.csv', index = False)
only_ap.to_csv('./score_maximize_Only_in_AlphaPose.csv', index = False)
same_opap_ap.to_csv('./score_maximize_Same_with_APOP_ver_ap.csv', index = False)




same_AP = settings.csv2list("score_maximize_Same_with_APOP_ver_ap.csv")
same_OP = settings.csv2list("score_maximize_Same_with_APOP_ver_op.csv")

same_AP = sorted(same_AP, key = lambda pers:pers[0])
same_AP = sorted(same_AP, key = lambda pers:pers[1])
same_OP = sorted(same_OP, key = lambda pers:pers[0])
same_OP = sorted(same_OP, key = lambda pers:pers[1])

In [64]:
print(same_OP[0])
print('--------------')
print(same_AP[0])

['193245', '1200575', '0.560367', '0.8647767553159352', '[117.879, 192.576, 1, 119.915, 188.008, 1, -1, -1, 0, 120.931, 191.057, 1, 136.17, 197.153, 1, 106.702, 214.427, 1, 141.263, 217.981, 1, 87.9018, 230.18, 1, 139.73, 240.854, 1, 88.9188, 215.957, 1, 130.586, 249.491, 1, 107.205, 266.772, 1, 130.585, 268.8, 1, 97.5564, 313.522, 1, 131.6, 313.515, 1, 94.5013, 359.248, 1, 131.092, 351.63, 1]']
--------------
['193245', '1200575', '0.9358564044865197', '0.8483905051375059', '[118.78889465332031, 191.9726104736328, 0.8900708556175232, 118.78889465332031, 190.07424926757812, 0.9312101602554321, 120.687255859375, 190.07424926757812, 0.72234046459198, 120.687255859375, 193.8709716796875, 0.8998424410820007, 135.87416076660156, 195.7693328857422, 0.8781397342681885, 105.50035095214844, 214.7529754638672, 0.8676959276199341, 141.56924438476562, 220.44805908203125, 0.889783501625061, 88.41508483886719, 229.93988037109375, 0.9088687896728516, 137.77252197265625, 245.1267852783203, 0.850532889

In [65]:
new_arr = []
def str2list(org):
    lists1= list(org.split(','))
    lists1[0] = lists1[0].replace("[", "")
    lists1[-1] = lists1[-1].replace("]","")

    return lists1
lis = str2list(same_OP[0][4])
lis2 = str2list(same_AP[0][4])
print(lis)
print(lis2)    

['117.879', ' 192.576', ' 1', ' 119.915', ' 188.008', ' 1', ' -1', ' -1', ' 0', ' 120.931', ' 191.057', ' 1', ' 136.17', ' 197.153', ' 1', ' 106.702', ' 214.427', ' 1', ' 141.263', ' 217.981', ' 1', ' 87.9018', ' 230.18', ' 1', ' 139.73', ' 240.854', ' 1', ' 88.9188', ' 215.957', ' 1', ' 130.586', ' 249.491', ' 1', ' 107.205', ' 266.772', ' 1', ' 130.585', ' 268.8', ' 1', ' 97.5564', ' 313.522', ' 1', ' 131.6', ' 313.515', ' 1', ' 94.5013', ' 359.248', ' 1', ' 131.092', ' 351.63', ' 1']
['118.78889465332031', ' 191.9726104736328', ' 0.8900708556175232', ' 118.78889465332031', ' 190.07424926757812', ' 0.9312101602554321', ' 120.687255859375', ' 190.07424926757812', ' 0.72234046459198', ' 120.687255859375', ' 193.8709716796875', ' 0.8998424410820007', ' 135.87416076660156', ' 195.7693328857422', ' 0.8781397342681885', ' 105.50035095214844', ' 214.7529754638672', ' 0.8676959276199341', ' 141.56924438476562', ' 220.44805908203125', ' 0.889783501625061', ' 88.41508483886719', ' 229.93988037

In [66]:

for i in range(len(same_OP)):
    new_arr.append(same_OP[i][0])
    new_arr.append(same_OP[i][1])
    op_key_list = str2list(same_OP[i][4])
    ap_key_list = str2list(same_AP[i][4])
    new_keypoint = []

    for j in range(len(op_key_list)):
        weight_op = float(same_OP[i][2])
        weight_ap = float(same_AP[i][2])
        weight_o = weight_op / (weight_op + weight_ap)
        weight_a = weight_ap / (weight_op + weight_ap)
        new_keypoint.append( (float(op_key_list[j]) * weight_o + float(ap_key_list[j]) *weight_a))
        if i ==0:
            print(weight_o)
            print()
    new_arr.append(new_keypoint)
    oks = settings.computeMaxOks(cocoEval, int(same_OP[i][0]), int(same_OP[i][1]), new_keypoint)
    new_arr.append(oks)
print(new_arr[0])

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743

0.3745209427413743



In [67]:
print(new_arr[2])
new_arr = np.array(new_arr)
new_arr = new_arr.reshape(-1, 4)
new_arr = new_arr.tolist()
new_df = pd.DataFrame(new_arr, columns = ["image_id", "gt_id", "keypoints", "oks"])
new_df["category_id"] = 1
new_df.to_csv("./score_maximize_comb_of_apop.csv", index = False)


[118.44812004996346, 192.19859248788813, 0.9312416224064011, 119.2106446893849, 189.3003956439461, 0.9569733958875959, 75.11283007531107, 118.51294129828443, 0.45180883281274936, 120.77854314470957, 192.8170803533634, 0.9373535444706447, 135.98495875519384, 196.28754519781427, 0.9237789558727809, 105.95039368639408, 214.6308908258291, 0.9172465735362394, 141.45454944907397, 219.52409378883027, 0.9310618885020965, 88.22284891711979, 230.02981020088242, 0.9429993364777378, 138.50563848885258, 243.52653770775234, 0.9065114525495688, 88.60373671588778, 215.2039078682234, 0.932831452859662, 125.58192802437696, 242.01172483615886, 0.5076189598368199, 106.138777720593, 268.6693978973219, 0.8464907300382972, 130.33110375748495, 269.4289263692014, 0.8639616037095663, 97.77562947043434, 313.4881396851271, 0.9171592230824361, 131.8986276918669, 313.4855180385279, 0.9271346971008194, 94.25666018326629, 359.1107667491176, 0.9074536295314375, 131.70837105295428, 353.88287676420896, 0.926313573009902

In [68]:
def list2dict(go2dic):
    whole_list= []
    for i in range(len(go2dic)):
        line_dic = {}
        line_dic["image_id"] = int(go2dic[i][0])
        line_dic["category_id"] = 1
        line_dic["score"] = float(go2dic[i][3])
        line_dic["gt_id"] = int(go2dic[i][1])
        lists1= list(go2dic[i][4].split(','))
        lists1[0] = lists1[0].replace("[", "")
        lists1[-1] = lists1[-1].replace("]","")
        for j in range(len(lists1)):
            lists1[j] = float(lists1[j])
        line_dic["keypoints"] = lists1
        whole_list.append(line_dic)
    return whole_list

same_in_apop = settings.csv2list('score_maximize_comb_of_apop.csv')
only_op = settings.csv2list('score_maximize_Only_in_OpenPose.csv')
only_ap = settings.csv2list('score_maximize_Only_in_AlphaPose.csv')
print(len(only_ap))
print(len(only_op))
same_list= []
for i in range(len(same_in_apop)):
    line_dic = {}
    line_dic["image_id"] = int(same_in_apop[i][0])
    line_dic["category_id"] = 1
    line_dic["score"] = float(same_in_apop[i][3])
    line_dic["gt_id"] = int(same_in_apop[i][1])
    lists1= list(same_in_apop[i][2].split(','))
    lists1[0] = lists1[0].replace("[", "")
    lists1[-1] = lists1[-1].replace("]","")
    for j in range(len(lists1)):
        lists1[j] = float(lists1[j])
    line_dic["keypoints"] = lists1
    same_list.append(line_dic)
    
oplist = list2dict(only_op)
aplist = list2dict(only_ap)

total_list=[]
for i in range(len(same_list)):
    total_list.append(same_list[i])
for i in range(len(oplist)):
    total_list.append(oplist[i])
for i in range(len(aplist)):
    total_list.append(aplist[i])
print(len(total_list))
with open('score_maximize_total_tuning_list.json', 'w') as make_file:
    make_file.write(json.dumps(total_list))

479
126
5970
